In [0]:
import numpy as np
import pandas as pd
import random
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Daniel Encarncao
Fin 410 Project

# Project Summary & Business Understanding 
We are going to train test and validate a neural network with a portion of a data set of German Credit Applicants. The data's last colunm is a yes or no column that determines whether the customers are good borrowers or not. Once we start running our Machine Learning models we will begin to identify the stronger and weaker indicators of good customers and try to improve our Machine Learning's predictability of good customers. We want to focus on increasing the accuracy of our machine learning model to be able to mitgate the risk of giving customers loans that they might not be able to repay. Having an accurate system in palce that can establish a customer's ability to pay back their debt more accuretulely decreases the risk of the default for the lender and makes the bank's profitability to risk ratio higher.  


In [0]:
from google.colab import files
uploaded = files.upload()

Saving MGMT635_GermanCreditData.csv to MGMT635_GermanCreditData.csv


Import setting looks different becasue I use google colab as my python notebook

In [0]:
CreditData = pd.read_table("MGMT635_GermanCreditData.csv", sep = ',')
data = CreditData
X = data.iloc[:,0:20].copy()
y = data.iloc[:,-1].copy()
n = len(y)
for i in range(n):
    if y[i] == 2:
        y[i] = 1
    elif y[i] == 1:
        y[i] = 0

In [0]:
y = y.astype(float)
X = X.astype(float)

regr = sm.Logit(y, X, family = sm.families.Binomial()).fit()
print(regr.summary2())

Optimization terminated successfully.
         Current function value: 0.484342
         Iterations 6
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.207     
Dependent Variable: 1.1              AIC:              1007.7153 
Date:               2020-04-13 19:44 BIC:              1105.8504 
No. Observations:   999              Log-Likelihood:   -483.86   
Df Model:           19               LL-Null:          -610.51   
Df Residuals:       979              LLR p-value:      6.6397e-43
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
-------------------------------------------------------------------
           Coef.    Std.Err.      z      P>|z|     [0.025    0.975]
-------------------------------------------------------------------
11        -0.5823     0.0697   -8.3501   0.0000   -0.7190   -0.4456
6          0.0259     0.0086    3.0119   0.0026    0.0090 

In [0]:
pred = regr.predict()
print(pred[0:10])

[0.56261154 0.02839812 0.4600665  0.57201153 0.14401504 0.06723001
 0.52450374 0.04202592 0.48533292 0.47273851]


In [0]:
def contrast(variable):
    level = list(set(variable))
    n = len(level)
    df = pd.DataFrame(np.zeros([n,n]), index = level, columns = level)
    for i in range(n):
        for j in range(n):
            df.iloc[i,j] = 1 if i==j else 0
    
    return df

In [0]:
print(contrast(data['1.1']))

result = [2 if r > .5 else 1 for r in pred]
print(confusion_matrix(y_pred = result, y_true = data['1.1']))

     1    2
1  1.0  0.0
2  0.0  1.0
[[621  78]
 [159 141]]


In [0]:
def conf_mat(y_true, y_pred): # function version
    df = contrast(y_true) # columns true, rows pred
    name = df.columns.tolist()
    n = len(y_pred)
    for j in range(len(name)):
        df.iloc[j,j] = 0
    for i in range(n):
        if y_true.iloc[i] == name[0]:
            if y_pred[i] == name[0]:
                df.iloc[0,0] += 1
            else :
                df.iloc[1,0] += 1
        else :
            if y_pred[i] == name[0]:
                df.iloc[0,1] += 1
            else :
                df.iloc[1,1] += 1
                
    return df

In [0]:
conf = conf_mat(data['1.1'], result)
print(conf, '\n')
print((conf.iloc[0,0]+conf.iloc[1,1])/len(y), '\n')
print(np.mean(pred))

       1      2
1  621.0  159.0
2   78.0  141.0 

0.7627627627627628 

0.3002648926673911


In [0]:
df = pd.DataFrame(data)
print(data)
df = df.drop(['43', '4.1', '152', '173', '192'], axis=1)

     11   6  34  43  1169  65  75  4  ...  143  152  2  173  1  192  201  1.1
0    12  48  32  43  5951  61  73  2  ...  143  152  1  173  1  191  201    2
1    14  12  34  46  2096  61  74  2  ...  143  152  1  172  2  191  201    1
2    11  42  32  42  7882  61  74  2  ...  143  153  1  173  2  191  201    1
3    11  24  33  40  4870  61  73  3  ...  143  153  2  173  2  191  201    2
4    14  36  32  46  9055  65  73  2  ...  143  153  1  172  2  192  201    1
..   ..  ..  ..  ..   ...  ..  .. ..  ...  ...  ... ..  ... ..  ...  ...  ...
994  14  12  32  42  1736  61  74  3  ...  143  152  1  172  1  191  201    1
995  11  30  32  41  3857  61  73  4  ...  143  152  1  174  1  192  201    1
996  14  12  32  43   804  61  75  4  ...  143  152  1  173  1  191  201    1
997  11  45  32  43  1845  61  73  4  ...  143  153  1  173  1  192  201    2
998  12  45  34  41  4576  62  71  3  ...  143  152  1  173  1  191  201    1

[999 rows x 21 columns]


In [0]:
train = df.iloc[0:979,:].copy()
test = df.iloc[979:999,:].copy()

X = train.iloc[:,0:15].copy()
y = train.iloc[:,-1].copy()

X_test = test.iloc[:,0:15].copy()
y_test = test.iloc[:,-1].copy()

n = len(y)
for i in range(n):
    if y[i] == 2:
        y[i] = 1
    elif y[i] == 1:
        y[i] = 0

y = y.astype(float)
X = X.astype(float)
X_test = X_test.astype(float)


In [0]:
regr = sm.Logit(y, X, family=sm.families.Binomial()).fit()
print(regr.summary2())

Optimization terminated successfully.
         Current function value: 0.483625
         Iterations 6
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.211     
Dependent Variable: 1.1              AIC:              976.9369  
Date:               2020-04-13 19:44 BIC:              1050.2349 
No. Observations:   979              Log-Likelihood:   -473.47   
Df Model:           14               LL-Null:          -599.97   
Df Residuals:       964              LLR p-value:      6.8610e-46
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
-------------------------------------------------------------------
           Coef.    Std.Err.      z      P>|z|     [0.025    0.975]
-------------------------------------------------------------------
11        -0.5925     0.0700   -8.4595   0.0000   -0.7298   -0.4553
6          0.0261     0.0087    2.9973   0.0027    0.0090 

In [0]:
pred = regr.predict(X_test)
result = [2 if r > .5 else 1 for r in pred]
conf = conf_mat(y_test, result)
print(conf, '\n')
print((conf.iloc[0,0]+conf.iloc[1,1])/np.sum(conf.values))

      1    2
1  12.0  1.0
2   4.0  3.0 

0.75


Removing some of the less effective predictors imrpove the models accuricy by 5%

In [0]:
df1 = data.drop(['43', '75', '101', '4.1', '67', '152', '2', '173', '192'], axis=1)

train = df1.iloc[0:979,:].copy()
test = df1.iloc[979:999,:].copy()

X = train.iloc[:,0:11].copy()
y = train.iloc[:,-1].copy()

X_test = test.iloc[:,0:11].copy()
y_test = test.iloc[:,-1].copy()

n = len(y)
for i in range(n):
    if y[i] == 2:
        y[i] = 1
    elif y[i] == 1:
        y[i] = 0

y = y.astype(float)
X = X.astype(float)
X_test = X_test.astype(float)

In [0]:
regr = sm.Logit(y, X, family=sm.families.Binomial()).fit()

Optimization terminated successfully.
         Current function value: 0.492238
         Iterations 6


In [0]:
pred = regr.predict(X_test)
result = [2 if r > .5 else 1 for r in pred]
conf = conf_mat(y_test, result)
print(conf, '\n')
print((conf.iloc[0,0]+conf.iloc[1,1])/np.sum(conf.values))

      1    2
1  12.0  2.0
2   4.0  2.0 

0.7


#Conclusion 

After splitting the data into training and testing sets based on the assigment requirements, I began to look at the results of our models to remove the weaker predictors. These are ususally the data sets that are less correlated to the value we are predicitng and therefore have less significance in determining the value, so we remove some from the data set as well as some other changes to help imrpove our model. 